# Data importation

In [1]:
import random
import numpy as np
from sklearn import svm
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
import pandas as pd
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import networkx as nx
import re

%matplotlib inline

In [2]:
info = pd.read_csv(
    "node_information.csv", 
    header= None, 
    names=["Id", "year", "title", "authors", "journal", "abstract"],
    sep=",",
    index_col = 0
)

In [3]:
X_train = pd.read_csv("training_set.txt", sep=" ", header=None)
X_test = pd.read_csv("testing_set.txt", sep=" ", header=None)
y_train = X_train[2]
X_train.drop([2], axis = 1, inplace = True)

print info.shape, X_train.shape, X_test.shape

(27770, 5) (615512, 2) (32648, 2)


In [4]:
######################
### FOR VALIDATION ###
######################


#####################
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2)
#####################

#####################
small_portion_to_train = 50000
small_portion_to_test  = 5000
#X_train = X_train[:small_portion_to_train]
#y_train = y_train[:small_portion_to_train]

#X_test  = X_test[:small_portion_to_test]
#y_test = y_test[:small_portion_to_test]
#####################

# Feature Preprocess

- list_authors is the list of authors in the papers
- list_universities is the list where the authors are from

In [5]:
def universities_to_keep(authors, universities):
    while('(' in authors and ')' in authors):
        universities.append( authors[authors.find('(')+1 : authors.find(')')] )
        authors = authors[: authors.find('(')] + authors[ authors.find(')')+1 : ]
            
    if '(' in authors:
        universities.append( authors[authors.find('(')+1 : ])
        authors = authors[: authors.find('(')]
    
    return authors, universities


def name_to_keep(author):
    if len(author.split(' ')) <= 1:
        return author
    
    while( author[0] == ' ' and len(author) > 0):
        author = author[1:]
    while( author[-1] == ' ' and len(author) > 0):
        author = author[:-1]
    
    author = author.replace('.', '. ')
    author = author.replace('.  ', '. ')
    name_to_keep = author.split(' ')[0][0] + '. ' + author.split(' ')[-1]

    return name_to_keep

# Transform concatenated names of authors to a list of authors 
list_authors = []
list_universities = []

info['authors'] = info['authors'].replace(np.nan, 'missing')
for authors in info['authors']:
    if authors != 'missing':
        ### split the different authors
        authors = authors.lower()
        
        ### Find the universities included in the name
        universities = []
        authors, universities = universities_to_keep(authors, universities)
        
        ### Split the authors
        authors = re.split(',|&', authors)
        
        ### For each author, check if university, and store it. Also, keep just the names (To be improved)
        authors_in_article = []      
        for author in authors:
            if author != ' ':
                authors_in_article.append(name_to_keep(author))
            
        list_universities.append(universities)
        list_authors.append(authors_in_article)
    else:
        list_universities.append(['missing'])
        list_authors.append(['missing'])   
        
info['authors'] = list_authors
info['universities'] = list_universities

# Topologic features

In [6]:
def make_graph(X_train, y_train, X_test):
    X_train = pd.concat([X_train, y_train], axis = 1)
    X_train = X_train.values
    G = nx.DiGraph()
    for i in range(X_train.shape[0]):
        source = X_train[i,0]
        target = X_train[i,1]
        G.add_node(source)
        G.add_node(target)
        if X_train[i,-1] == 1:
            G.add_edge(source,target)
            
    X_test = X_test.values
    for i in range(X_test.shape[0]):
        source = X_test[i,0]
        target = X_test[i,1]
        G.add_node(source)
        G.add_node(target)
        
    return G  

In [7]:
G = make_graph(X_train, y_train, X_test)  

In [8]:
def create_topologic_features(X, G):
    X_ = X.copy()
    X = X.values
    
    X_['Betweeness centrality'] = compute_betweeness_array(X, G)
    X_['Number common neighbours'] = make_common_neighbors(X, G)
    X_['Jaccard coefficienf'] = make_jaccard(X, G)
    diff_deg, to_deg = compute_diff_inlinks(X, G)
    X_['Difference in inlinks coefficient'] = diff_deg
    X_["Number of times to cited"] = to_deg
    X_['Same cluster'] = same_community(X,G)
    return X_

In [9]:
%%time
from create_topologic_features import create_topologic_features
X_train = create_topologic_features(X_train, G)
X_test = create_topologic_features(X_test, G)

CPU times: user 1min 22s, sys: 752 ms, total: 1min 23s
Wall time: 1min 23s


# Semantic features
- Cosine similarity within the titles as tf-idf
- Cosine similarity within the abstracts as tf-idf
- Cosine similarity within the titles as word2vec
- Cosine similarity within the abstracts as word2vec

### To try
- Difference cosine similarities?
- Keep the stopwords or not?
- Stemmise the words of not?

In [10]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

def text_to_wordlist(text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(text).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[\W]"," ", text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("punkt")   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def text_to_sentences(text, tokenizer, remove_stopwords=False ):
    # Function to split a text into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(text_to_wordlist(raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

train = False
sentences = []  # Initialize an empty list of sentences
word_list_abstract=pd.Series(index=info.index,dtype=np.str)
word_list_title=pd.Series(index=info.index,dtype=np.str)
print "Parsing sentences from training set"
for idx in info.index:
    w_list_a = text_to_sentences(info.loc[idx,"abstract"], tokenizer, True)
    sentences += w_list_a
    word_list_abstract[idx]= w_list_a
    w_list_t = text_to_sentences(info.loc[idx,"title"], tokenizer, True)
    sentences += w_list_t
    word_list_title[idx]= w_list_t

from gensim.models import word2vec
from gensim.models import Word2Vec
num_features = 200    # Word vector dimensionality  
if train:
    # Import the built-in logging module and configure it so that Word2Vec 
    # creates nice output messages
    import logging
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # Set values for various parameters
                    
    min_word_count = 10   # Minimum word count                        
    num_workers = 4       # Number of threads to run in parallel
    context = 10          # Context window size                                                                                    
    downsampling = 1e-3   # Downsample setting for frequent words

    # Initialize and train the model (this will take some time)

    print "Training model..."
    model = word2vec.Word2Vec(
        sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling
    )

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()
    model_name = "200features_10minwords_10context"
    model.save(model_name)
else:
    model = Word2Vec.load("200features_10minwords_10context")

def makeFeatureVec(list_of_words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in list_of_words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(text_list, model, num_features):
    # Given a set of texts (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    textFeatureVecs = np.zeros((len(text_list),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for list_of_words in text_list:
        # Print a status message every 1000th review
        """
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(text_list))
        """
        # Call the function (defined above) that makes average feature vectors
        textFeatureVecs[counter] = makeFeatureVec(list_of_words, model, num_features)

        # Increment the counter
        counter = counter + 1.
    return textFeatureVecs

centroid_title = dict(zip(info.index, getAvgFeatureVecs(info.title.values, model, num_features)))
centroid_abstract = dict(zip(info.index, getAvgFeatureVecs(info.abstract.values, model, num_features)))

from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

# Pre-allocate an array for the training set bags of centroids (for speed)
bag_centroids_abstract = {}
#centroids_title = {}
# Transform the training set reviews into bags of centroids
for review, idx in zip(word_list_abstract, word_list_abstract.index):
    bag_centroids_abstract[idx] = create_bag_of_centroids(review[0], word_centroid_map)
#for review, idx in zip(word_list_title, word_list_title.index):
#    centroids_title[idx] = create_bag_of_centroids( review[0], word_centroid_map )

from scipy.spatial.distance import cosine
def compute_cosines(centroids_dic, X):
    bag=[cosine(centroids_dic[x[0]], centroids_dic[x[1]]) for x in X]
    return np.array(bag)

def create_nlp_features(X, centroid_title, centroid_abstract, bag_centroids_abstract):
    X_ = X.copy()
    X = X.values
    
    X_['CosineD_title_centroid'] = compute_cosines(centroid_title,X)
    X_['CosineD_abstract_centroid'] = compute_cosines(centroid_abstract,X)
    X_['CosineD_abstract_bag_centroids'] = compute_cosines(bag_centroids_abstract,X)
    
    return X_


[nltk_data] Downloading package punkt to /home/bat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Parsing sentences from training set

/home/bat/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/bat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:146: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future



Time taken for K Means clustering:  57.7536728382 seconds.


In [11]:
%%time
X_train = create_nlp_features(X_train, centroid_title, centroid_abstract, bag_centroids_abstract)
X_test = create_nlp_features(X_test,centroid_title, centroid_abstract, bag_centroids_abstract)

CPU times: user 1min 22s, sys: 54.7 ms, total: 1min 22s
Wall time: 1min 22s


# Attribute features

- Difference in publication year
- Number of common authors
- Self-citation
- Same journal
- Number of times "to" cited (Attraction of the "to" paper)

### To try
- Number of times each author of "to" cited [Sum of these number of times] ?
- Number of times each journal cited?
- Number of same university??

In [12]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

def text_to_wordlist(text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(text).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[\W]"," ", text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("punkt")   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def text_to_sentences(text, tokenizer, remove_stopwords=False ):
    # Function to split a text into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(text_to_wordlist(raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

train = False
sentences = []  # Initialize an empty list of sentences
word_list_abstract=pd.Series(index=info.index,dtype=np.str)
word_list_title=pd.Series(index=info.index,dtype=np.str)
print "Parsing sentences from training set"
for idx in info.index:
    w_list_a = text_to_sentences(info.loc[idx,"abstract"], tokenizer, True)
    sentences += w_list_a
    word_list_abstract[idx]= w_list_a
    w_list_t = text_to_sentences(info.loc[idx,"title"], tokenizer, True)
    sentences += w_list_t
    word_list_title[idx]= w_list_t

from gensim.models import word2vec
from gensim.models import Word2Vec
num_features = 200    # Word vector dimensionality  
if train:
    # Import the built-in logging module and configure it so that Word2Vec 
    # creates nice output messages
    import logging
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # Set values for various parameters
                    
    min_word_count = 10   # Minimum word count                        
    num_workers = 4       # Number of threads to run in parallel
    context = 10          # Context window size                                                                                    
    downsampling = 1e-3   # Downsample setting for frequent words

    # Initialize and train the model (this will take some time)

    print "Training model..."
    model = word2vec.Word2Vec(
        sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling
    )

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()
    model_name = "200features_10minwords_10context"
    model.save(model_name)
else:
    model = Word2Vec.load("200features_10minwords_10context")

def makeFeatureVec(list_of_words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in list_of_words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(text_list, model, num_features):
    # Given a set of texts (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    textFeatureVecs = np.zeros((len(text_list),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for list_of_words in text_list:
        # Print a status message every 1000th review
        """
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(text_list))
        """
        # Call the function (defined above) that makes average feature vectors
        textFeatureVecs[counter] = makeFeatureVec(list_of_words, model, num_features)

        # Increment the counter
        counter = counter + 1.
    return textFeatureVecs

centroid_title = dict(zip(info.index, getAvgFeatureVecs(info.title.values, model, num_features)))
centroid_abstract = dict(zip(info.index, getAvgFeatureVecs(info.abstract.values, model, num_features)))

from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

# Pre-allocate an array for the training set bags of centroids (for speed)
bag_centroids_abstract = {}
#centroids_title = {}
# Transform the training set reviews into bags of centroids
for review, idx in zip(word_list_abstract, word_list_abstract.index):
    bag_centroids_abstract[idx] = create_bag_of_centroids(review[0], word_centroid_map)
#for review, idx in zip(word_list_title, word_list_title.index):
#    centroids_title[idx] = create_bag_of_centroids( review[0], word_centroid_map )

from scipy.spatial.distance import cosine
def compute_cosines(centroids_dic, X):
    bag=[cosine(centroids_dic[x[0]], centroids_dic[x[1]]) for x in X]
    return np.array(bag)

def create_nlp_features(X, centroid_title, centroid_abstract, bag_centroids_abstract):
    X_ = X.copy()
    X = X.values
    
    X_['CosineD_title_centroid'] = compute_cosines(centroid_title,X)
    X_['CosineD_abstract_centroid'] = compute_cosines(centroid_abstract,X)
    X_['CosineD_abstract_bag_centroids'] = compute_cosines(bag_centroids_abstract,X)
    
    return X_


[nltk_data] Downloading package punkt to /home/bat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Parsing sentences from training set
Time taken for K Means clustering:  81.2030808926 seconds.


/home/bat/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:146: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [13]:
%%time
X_train = create_nlp_features(X_train, centroid_title, centroid_abstract, bag_centroids_abstract)
X_test = create_nlp_features(X_test,centroid_title, centroid_abstract, bag_centroids_abstract)

CPU times: user 1min 17s, sys: 63.2 ms, total: 1min 17s
Wall time: 1min 17s


In [14]:
%%time
from create_attribute_features import create_attribute_features
X_train = create_attribute_features(X_train,info)
X_test = create_attribute_features(X_test,info)

CPU times: user 3min 59s, sys: 1.41 s, total: 4min 1s
Wall time: 4min


# Author Graph features

In [15]:
%%time 
from author_graph import make_graph_authors, create_topologic_features_authors

G_authors = make_graph_authors(X_train, y_train, info)
X_train = create_topologic_features_authors(X_train, G_authors, info, betweeness = True, common_neigh_and_jacc = True, inlinks = True)
X_test = create_topologic_features_authors(X_test, G_authors, info,  betweeness = True, common_neigh_and_jacc = True, inlinks = True)

CPU times: user 9min 55s, sys: 1.72 s, total: 9min 56s
Wall time: 10min 35s


In [16]:
X_train.head()

,0,1,Betweeness centrality,Number common neighbours,Jaccard coefficienf,Difference in inlinks coefficient,Number of times to cited,Same cluster,CosineD_title_centroid,CosineD_abstract_centroid,...,Self citation,Same journal,Authors betweeness,Authors common neighbors,Authors jaccard,Authors max difference in inlinks,Authors sum difference in inlinks,Authors max of times to cited,Authors sum of times to cited,Authors of times to cited
3432,9119,9810126,-0.005150,1,0.006098,143,143,1,0.012299,0.001006,...,0,0,-0.105209,200,0.130890,720,1910,2258,872,522
116401,4115,9912072,-0.008391,32,0.105611,254,275,1,0.003831,0.000732,...,0,0,-0.139588,978,0.397399,1270,1749,3376,2095,749
244993,11079,9802068,-0.005690,7,0.028926,158,186,1,0.003094,0.000562,...,0,0,-0.099575,424,0.268695,685,1137,1688,893,844
485454,201004,108119,0.000360,10,0.109890,10,25,1,0.005318,0.000895,...,0,0,-0.001035,151,0.177022,64,-246,326,258,163
470066,9210040,112110,0.000000,0,0.000000,-1,4,0,0.007626,0.000821,...,0,0,1.023226,362,0.058125,-4972,-4972,180,180,180


In [17]:
X_test.head()

,0,1,Betweeness centrality,Number common neighbours,Jaccard coefficienf,Difference in inlinks coefficient,Number of times to cited,Same cluster,CosineD_title_centroid,CosineD_abstract_centroid,...,Self citation,Same journal,Authors betweeness,Authors common neighbors,Authors jaccard,Authors max difference in inlinks,Authors sum difference in inlinks,Authors max of times to cited,Authors sum of times to cited,Authors of times to cited
152793,211086,9504102,-0.002521,9,0.079646,90,90,1,0.012404,0.000968,...,0,0,-0.095550,40,0.053333,479,700,703,482,351.5
325283,7236,9606017,-0.001621,5,0.036765,75,80,0,0.012277,0.000500,...,0,0,-0.994251,725,0.116634,4744,4518,5152,5152,5152.0
54796,9305090,9312034,0.000396,0,0.000000,-11,1,0,0.010361,0.000580,...,0,0,0.949638,1185,0.189904,-4351,-3947,1205,801,201.0
48423,10216,9301100,-0.000504,0,0.000000,17,17,0,0.009030,0.001801,...,0,0,-0.064275,21,0.032159,382,579,599,402,299.5
465770,9806193,9909222,-0.000324,0,0.000000,-2,7,0,0.011871,0.000610,...,0,1,-0.035989,4,0.010667,187,173,232,232,232.0


In [35]:
X_train=X_train.drop([0,1], axis = 1) 
X_test = X_test.drop([0,1], axis = 1)

In [18]:
path = "validation_set2/"
X_train.to_csv(path+"X_train.csv")
X_test.to_csv(path+"X_test.csv")
y_train.to_csv(path+"y_train.csv")
y_test.to_csv(path+"y_test.csv")

# Classifier

In [23]:
def score(pred, real):
    tot = 0
    for i, val in enumerate(real):
        if pred[i] == val:
            tot += 1
    return float(tot)/len(real)

In [31]:
from sklearn.base import BaseEstimator
from sknn.mlp import Layer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sknn.mlp import Classifier as MLP
from sklearn.pipeline import Pipeline

scaler1=StandardScaler()
scaler = MinMaxScaler(feature_range=(0.0, 1.0))
nn = MLP(
layers=[
Layer("Rectifier", units=2000),
        Layer("Softmax")],
        learning_rule = 'momentum', learning_rate=0.02, batch_size = 150,dropout_rate = 0.25,
        n_iter=50,
        verbose = 1, 
        valid_size = 0.1, 
        n_stable = 15,
        debug = False,
        #    regularize = 'L2'
)


In [33]:
clf = Pipeline([
    ("scaler", scaler1),
    ('neural network', nn)
    ])

clf.fit(X_train.drop([0,1], axis = 1), y_train)
pred = clf.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(clf.predict(X_train.drop([0,1], axis = 1)),y_train)

KeyboardInterrupt: 

In [29]:
%%time 
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100,n_jobs=2)
rfc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = rfc.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(rfc.predict(X_train.drop([0,1], axis = 1)),y_train)

0.958546907874
0.999987815007
CPU times: user 3min 13s, sys: 999 ms, total: 3min 14s
Wall time: 1min 38s


In [20]:
%%time 
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_estimators = 400,n_jobs=2)
etc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = etc.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(etc.predict(X_train.drop([0,1], axis = 1)),y_train)

0.961536274502
1.0
CPU times: user 9min 33s, sys: 4.21 s, total: 9min 38s
Wall time: 4min 52s


In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators = 200)
gbc.fit(X_train.drop([0,1], axis = 1), y_train)
pred = gbc.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(gbc.predict(X_train.drop([0,1], axis = 1)),y_train)

In [30]:
%%time 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
clf =  Pipeline([
    ("scaler", MinMaxScaler()),
    ("regression",LogisticRegression(C=200000,max_iter = 1000))
    ])
clf.fit(X_train.drop([0,1], axis = 1), y_train)
pred = clf.predict(X_test.drop([0,1], axis = 1))
print score(pred, y_test)
print score(clf.predict(X_train.drop([0,1], axis = 1)),y_train)

0.952941845446
0.956489422411
CPU times: user 41 s, sys: 848 ms, total: 41.8 s
Wall time: 41.7 s


In [ ]:
def make_submission(predicted_label, name = 'submit.csv'):
    submit_d = d = {'id' : pd.Series(np.arange(1,X_test.shape[0]+1).astype(int)),
                    'category' : pd.Series(predicted_label).astype(int)}
    submit = pd.DataFrame(submit_d)
    submit.to_csv(name,index=False)
    return submit
